In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

## Indeed

In [2]:
# Site dictionary
site = "http://au.indeed.com/jobs?q=TERM&l=Sydney+NSW&start=PAGE"
page_iterator =  10
domain = "http://au.indeed.com"
term = "analyst"

In [25]:
page = 1
old_list_of_page_links, new_list_of_page_links, list_of_page_links = [], [], []
failed = []
noNew = False
job_desc=[]

while noNew == False:

    # Collecting a list of pages to scrape and saving their job titles and links
    result_number = page*page_iterator
    req = requests.get(site.replace('PAGE', str(result_number)))
    soup = BeautifulSoup(req.text, 'html.parser')
    div = soup.find_all('div', {'class':'title'})
    new_list_of_page_links = [domain + d.a['href'] for d in div]
    
    # Check whether there were any new
    if page == 50:
        noNew = True
    else:
        list_of_page_links +=[domain + d.a['href'] for d in div]
        old_list_of_page_links = new_list_of_page_links
    
        # Go to the next page of search results
        page += 1

print('Finished collecting the list of job descriptions.')

# Crawling through each page
for i,p in enumerate(list_of_page_links):
    try:
        req = requests.get(p)
        soup = BeautifulSoup(req.text,'html.parser')
        main_text = soup.find('div', {'id':'jobDescriptionText'})
        job_desc.append(main_text)
        time.sleep(1)
    except:
        failed.append(p)
    print('Finished Downloading: %i out of %i: %f [%s]'% (i+1, len(list_of_page_links), round(i/len(list_of_page_links)*100,3),'#'*int(i/len(list_of_page_links)*10)+' '*(1-int(len(list_of_page_links)-i/len(list_of_page_links))*10)),end='\r')

Finished collecting the list of job descriptions.


In [26]:
job_desc[0]

<div class="jobsearch-jobDescriptionText" id="jobDescriptionText"><p><b>Customer Service Assistant - Food Industry</b></p><p><b>spiralfoods.com.au</b></p><p>Spiral Foods is a leading Australian organic and natural foods company, with iconic brand such as BONSOY.</p><p>We’re looking for an experienced customer service staff member for a permanent position in our Coburg office.</p><p><b>We are looking for someone who is: </b></p><ul><li>Approachable, friendly, reliable, with a can-do attitude.</li><li>Comfortable with conducting customer communications, either face to face, phone or via email in a friendly and professional manner</li><li>Super organised and be able to work towards deadlines</li><li>Attention to details</li><li>Numerate and proficient with MS Office</li><li>Food lovers ideal but not required</li><li>SAP experience or equivalent preferred</li></ul><p><b>Key responsibilities of Customer Service Assistant: </b></p><ul><li>Taking inbound queries through phone, email &amp; web

In [64]:
import re

r = str(job_desc[0])
titles = re.findall('<b>(.+?)</b>', r)
content = re.findall('</b></p><p>(.+?)</p>', r)
d = {t:c for t,c in zip(titles, content)}

In [67]:
content

['<b>spiralfoods.com.au</b>']

In [ ]:
## Seek

In [ ]:
## Jora

In [ ]:
## Glassdoor

In [20]:
len(p)

17

In [ ]:
for s in d.keys():
    if s in site:
        page = 0
        site_dict = d[s]
        site_dict["TERM"] = term
        noNew = False
        old_list_of_page_links = []
        job_desc  = []
        
        while noNew == False:
        # Setting the required variables.
            site_dict["PAGE"] = str(page*site_dict["PAGE_ITERATOR"])

            # Loading the page
            pattern = re.compile("|".join(site_dict["replacements"]))
            site_search = pattern.sub(lambda m: site_dict[re.escape(m.group(0))],site_dict["site"])
            req = requests.get(site_search)
            
            # Collecting a list of pages to scrape and saving their job titles and links
            soup = BeautifulSoup(req.text, 'html.parser')
            div = soup.find_all('div', {'class':'title'})
            list_of_page_links = [site_dict["DOMAIN"] + d.a['href'] for d in div]
            
            # Check whether there were any new
            if list_of_page_links == old_list_of_page_links:
                noNew = True

            # Crawling through each page
            for p in list_of_page_links:
                req = requests.get(p)
                soup = BeautifulSoup(req.text,'html.parser')
                main_text = soup.find('div', {'id':'jobDescriptionText'})
                job_desc.append(main_text)

            # Go to the next page of search results
            page += 1
            
            # Setting up old list of page links
            old_list_of_page_links = list_of_page_links